In [ ]:
#%%capture
%pip install --upgrade opensearch-py 

In [ ]:
#Perform the necessary imports.
from opensearchpy import OpenSearch
from opensearchpy.helpers import scan

#Date
import datetime

#Password 
from getpass import getpass

#JSON
import json

In [ ]:
#User Variables
# You can use * as a wildcard. Don't adjust the keys
# Logon Types (Not exhaustive)
# 2 - Local Logon (At the terminal)
# 3 - Network Logon
# 10 - Remote Desktop
search_fields = {
    "UserName": "administrator", #A Username to search on (i.e. this username signed into X machines)
    "SourceIP": "", # A Source to search on (i.e. all users who signed in from X)
    "Computer": "", # A Target (i.e. all users who signed into X machine)
    "LogonType": "", # The type of logon (i.e. show me all RDP logons. Types above)
    "EventID": "" #The event ID type (i.e. only show me 4625 events (failed logons))
}

#Should be in ISO8601 - e.g. 2023-08-12T06:04:32.188Z
date_from = ""
date_to = ""

In [ ]:
hosts = [{"host": "os01", "port": 9200}] #Opensearch details
username = 'admin' #Username, there is always an admin account in DFIR2Go
password = getpass() #Prompt the user for the password 
auth = (username, password) #Create the authentication details

ca_certs_path = '/certs/ca/ca.crt' #Certs path

In [ ]:
client = OpenSearch(
    hosts=hosts, 
    http_compress=True,
    http_auth=auth,
    use_ssl=True,
    timeout=300,
    verify_certs=True,
    ssl_assert_hostname=False,
    ssl_show_warn=False,
    ca_certs=ca_certs_path        
)

In [ ]:
index = 'logstash-vr-rdpauth*' #The index you want to search.

should_clauses = []
for field, term in search_fields.items():
  if term:
    if "*" in term:
      should_clauses.append({ "bool": { "should": [ { "query_string": { "fields": [field], "query": term } } ], "minimum_should_match": 1 }})
    else:
      should_clauses.append({ "bool": { "should": [ { "match": {field: term} } ], "minimum_should_match": 1 } })

# Initialize filter_list 
filter_list = [ 
      { 
        "bool": { 
          "filter": should_clauses 
        } 
      }] 
      
# Conditionally add the range filter if dates are provided 
if date_from and date_to: 
  filter_list.append({ 
    "range": { 
      "@timestamp": { 
        "gte": date_from, 
        "lte": date_to, 
        "format": "strict_date_optional_time" 
      } 
    }})

#The query is the search to perform. The following will return an entire index. 
#To assist in building your query, you can use the developer tools in the browser. 
#When performing a search, review the submitted webpage which will contain a query value which can be used as a starting point.
query = {
  "sort": [
    {
      "@timestamp": {
        "order": "asc",
        "unmapped_type": "boolean"
      }
    }
  ],
    "size": "10000",  
    "timeout": "300s",
    "query": {
        "bool": {
            "must": [],
            "filter": should_clauses,
            "should": [],
            "must_not": []
          }
    }
}

#Use Scan function to interact with OpenSearch, once the results have been returned, loop through all of them printing the _source component.
for results in scan(client, query=query, index=index):
    event_id = results["_source"]["EventID"]

    #Now work through the various event IDs
    if event_id == 21:
      # Terminal Services - Successful Authentication
      print("Description")

    elif event_id == 22:
      # Terminal Services - Shell Start
      print("Description")

    elif event_id == 23:
      # Terminal Services - Log off
      print("Description")

    elif event_id == 24:
      # Terminal Services - Session Disconnect
      print("Description")

    elif event_id == 25:
      # Terminal Services - Reconnect to RDP Session
      print("Description")
      
    elif event_id == 39:
      # Terminal Services - Disconnected by another user
      print("Description")

    elif event_id == 40:
      # Terminal Services - Disconnected
      print("Description")

    elif event_id == 1194:
      # Remote Desktop - User auth successful
      print("Description")

    elif event_id == 4624:
      # Security - Successful Logon
      print("Successful Logon")

    elif event_id == 4625:
      # Security - Failed Logon
      print("Failed Logon")

    elif event_id == 4636:
      # Security - Account was logged off
      print("Description")

    elif event_id == 4647:
      # Security - User initiated log off
      print("Description")

    else:
      # Default action if EventID is not found 
      print("Default handler")
    